In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from google.colab import drive
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [0]:
import os
os.getcwd()
os.chdir('/content/MyDrive/My Drive/Rnn')

In [209]:
data=pd.read_csv('chennai_reservoir_levels.csv')
data=data.tail(2500)
data=data.reset_index()
data.head()

,index,Date,POONDI,CHOLAVARAM,REDHILLS,CHEMBARAMBAKKAM
0,3147,13-08-2012,849.0,85.0,1381.0,1231.0
1,3148,14-08-2012,837.0,85.0,1381.0,1259.0
2,3149,15-08-2012,832.0,85.0,1384.0,1274.0
3,3150,16-08-2012,832.0,85.0,1384.0,1283.0
4,3151,17-08-2012,825.0,85.0,1384.0,1289.0


In [0]:
from statsmodels.tsa.arima_model import ARMA, ARIMA

In [0]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error

In [0]:
data['label']=0

In [0]:
data['Avg']=0

In [219]:
for i in range(2500):
  data['Avg'][i] = (data['POONDI'][i]+data['CHOLAVARAM'][i]+data['REDHILLS'][i]+data['CHEMBARAMBAKKAM'][i] ) / 4.0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [220]:
for i in  range(2500):
  if data['Avg'][i]<=1000:
    data['label'][i]=1
  else:
    data['label'][i]=0
    
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [0]:
#divide into train and validation set
train = data.head(1000)
valid = data.tail(400)

In [0]:
# ARIMA example
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(train['Avg'], order=(4,1,0))

In [0]:
model_fit = model.fit(disp=1)

In [223]:
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             ARIMA Model Results                              
==============================================================================
Dep. Variable:                  D.Avg   No. Observations:                  999
Model:                 ARIMA(4, 1, 0)   Log Likelihood               -5208.783
Method:                       css-mle   S.D. of innovations             44.472
Date:                Sun, 11 Aug 2019   AIC                          10429.567
Time:                        02:06:31   BIC                          10459.007
Sample:                             1   HQIC                         10440.757
                                                                              
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.6348      2.576      0.246      0.805      -4.413       5.683
ar.L1.D.Avg    -0.1475      0.031     -4.741      0.000      -0.209      -0.087
ar.L2.D.Avg     0.1566      0.030      5.176      0.000       0.097       0.216
ar.L3.D.Avg     0.2699      0.030      8.924      0.000       0.211       0.329
ar.L4.D.Avg     0.1764      0.031      5.677      0.000       0.115       0.237
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.2437           -0.0000j            1.2437           -0.0000
AR.2           -0.4987           -1.5222j            1.6018           -0.3004
AR.3           -0.4987           +1.5222j            1.6018            0.3004
AR.4           -1.7767           -0.0000j            1.7767           -0.5000
-----------------------------------------------------------------------------
"""

In [0]:
start_index = valid.index.min()
end_index = valid.index.max()

#Predictions
predictions = model_fit.predict(start=start_index, end=end_index)

In [0]:
y = data.set_index('Date')
data=data.set_index('Date')

In [226]:
predictions[1:9]

1001   -0.383713
1002   -0.372494
1003   -0.098063
1004    0.026920
1005    0.158199
1006    0.234458
1007    0.325906
1008    0.381837
dtype: float64

In [0]:
mse=mean_squared_error(valid['Avg'],predictions) 

In [228]:
rmse = sqrt(mse)
print('RMSE: {}, MSE:{}'.format(rmse,mse))

RMSE: 378.15136327280436, MSE:142998.45354508047


In [229]:
predictions_ARIMA_diff = pd.Series(predictions, copy=True)
print (predictions_ARIMA_diff.head(4))

1000   -0.971737
1001   -0.383713
1002   -0.372494
1003   -0.098063
dtype: float64


In [230]:
predictions=list(predictions)
for i in range(len(predictions)):
  if predictions[i]<0.5:
    predictions[i]=0
  else:
    predictions[i]=1
print(predictions)    

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 